In [ ]:
import asyncio, json, xmltodict, time, pprint
from jikanpy import AioJikan
from pathlib import Path

data = Path('data')

In [1]:
# Load the XML file and convert to dict
with open('animelist.xml', 'r') as f:
	data = xmltodict.parse(f.read())['myanimelist']

def clean_anime(anime: dict):
	anime.update(anime['details']['data'])
	del anime['details']

	# Define key map for necessary fields and their corresponding types
	key_map = {
		# My data
		'my_watched_episodes': int,
		'my_start_date': None,
		'my_finish_date': None,
		'my_rated': None,
		'my_score': int,
		'my_storage': None,
		'my_storage_value': None,
		'my_status': None,
		'my_comments': None,
		'my_times_watched': int,
		'my_rewatch_value': None,
		'my_priority': None,
		'my_tags': None,
		'my_rewatching': int,
		'my_rewatching_ep': int,
		'my_discuss': int,
		'my_sns': None,

		# MAL data
		'mal_id': int,
		'url': None,
		'images': None,
		'trailer': None,
		'approved': None,
		'title': None,
		'title_english': None,
		'title_japanese': None,
		'title_synonyms': None,
		'type': None,
		'source': None,
		'episodes': None,
		'status': None,
		'airing': None,
		'aired': None,
		'duration': None,
		'rating': None,
		'score': None,
		'scored_by': None,
		'rank': None,
		'popularity': None,
		'members': None,
		'favorites': None,
		'synopsis': None,
		'background': None,
		'season': None,
		'year': None,
		'broadcast': None,
		'producers': None,
		'licensors': None,
		'studios': None,
		'genres': None,
		'explicit_genres': None,
		'themes': None,
		'demographics': None,
	}

	def clean_fields(data: dict, fields: dict):
		for field in list(data.keys()):  # Iterate over a copy of the keys
			if field not in fields:
				del data[field]
			elif isinstance(data[field], dict) and isinstance(fields[field], dict):
				clean_fields(data[field], fields[field])
			elif fields[field] is not None and data[field] is not None:
				data[field] = fields[field](data[field])

	clean_fields(anime, key_map)

	return anime

async def fetch_anime_details(data: dict):
	aio_jikan = AioJikan()

	animes = data['anime']
	n = len(animes)
	for i, anime in enumerate(animes):
		anime_id = int(anime['series_animedb_id'])
		details = await aio_jikan.anime(anime_id)
		anime['details'] = details
		anime = clean_anime(anime)
		# pprint.pprint(anime)
		print(f'{i+1}/{n}')
		time.sleep(1)  # Delay to ensure we don't exceed rate limit
	
	await aio_jikan.close()
	
	return animes

# Load the anime details
completed_data = asyncio.run(fetch_anime_details(data))

# Save to a JSON file
with open('animes_data.json', 'w') as f:
	json.dump(completed_data, f)


FileNotFoundError: [Errno 2] No such file or directory: 'animelist.xml'